In [1]:
pip install fake_useragent pandas 


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import concurrent.futures
import pandas as pd

In [3]:
url = 'https://www.macys.com/shop/womens-clothing/accessories?id=29440'

ua = UserAgent()
userAgent = ua.random
headers = {'User-Agent': userAgent}
page = requests.get(url, headers = headers)
soup = BeautifulSoup(page.content, 'html.parser')

sales_box = soup.find_all('div', class_ = "product-thumbnail-container")


In [26]:
product_brands = []
product_names = []
current_prices = []
old_prices = []
ratings = []
product_urls = []

def scrape_macy(url):

    ua = UserAgent()
    userAgent = ua.random
    headers = {'User-Agent': userAgent}
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.content, 'html.parser')

    sales_box = soup.find_all('div', class_ = "product-thumbnail-container")
    for box in sales_box:
        #product_brand
        if box.find('div', class_ = "product-brand medium") is not None:
            product_brand = box.find('div', class_ = "product-brand medium")
            product_brands.append(product_brand.text)
        else:
            product_brands.append('None')

        #product_names
        if box.find('div', class_ = "product-name medium") is not None:
            product_name = box.find('div', class_ = "product-name medium")
            product_names.append(product_name.text.lstrip())
        else:
            product_names.append('None')

            #current_prices
        if box.find('span', class_ = "discount price-reg") is not None:
            current_price = box.find('span', class_ = "discount price-reg")
            current_prices.append(current_price.text.rstrip())
        elif box.find('span', class_ = "price-reg") is not None:
            current_price = box.find('span', class_ = "price-reg")
            current_prices.append(current_price.text.rstrip())
        else:
            current_prices.append('None')

            #old_prices
        if box.find('span', class_ = "price-strike") is not None:
            old_price = box.find('span', class_ = "price-strike")
            old_prices.append(old_price.text.strip())
        else:
            old_prices.append('None')

        #ratings
        if box.select_one("div.rating > div > fieldset") is not None:
            rating = box.select_one("div.rating > div > fieldset")
            rating = rating['aria-label']
            ratings.append(rating.replace('Rated','').replace(' stars','').strip())
        else:
            ratings.append('None')

        #product_urls
        if box.find('a', class_ = "brand-and-name") is not None:
            product_url = box.find('a', class_ = "brand-and-name")
            product_url = product_url['href']
            product_urls.append('https://www.macys.com'+ product_url)
        else:
            product_urls.append('None')

    if soup.find('a', class_ = 'pagination-next') is None:
        a = 1
        b = 'e'
        a+b
    else:
        pass



base_url, query_string = url.split('?',1)
index = 1
while True:
    try:
        page = '/Pageindex/'+ '{}?{}'.format(index,query_string)
        index += 1
        new_url = base_url + page
        scrape_macy(new_url)
        print(new_url)
        print(len(product_urls))
    except Exception as e:
        print(e)
        break


data = pd.DataFrame({
    'Product Brand': product_brands,
    'Product Name': product_names,
    'Current Price': current_prices,
    'Old Price': old_prices,
    'Rating': ratings,
    'Product URL': product_urls
})
data


https://www.macys.com/shop/womens-clothing/accessories/Pageindex/1?id=29440
120
https://www.macys.com/shop/womens-clothing/accessories/Pageindex/2?id=29440
240
https://www.macys.com/shop/womens-clothing/accessories/Pageindex/3?id=29440
360
https://www.macys.com/shop/womens-clothing/accessories/Pageindex/4?id=29440
480
https://www.macys.com/shop/womens-clothing/accessories/Pageindex/5?id=29440
600
https://www.macys.com/shop/womens-clothing/accessories/Pageindex/6?id=29440
720
https://www.macys.com/shop/womens-clothing/accessories/Pageindex/7?id=29440
840
https://www.macys.com/shop/womens-clothing/accessories/Pageindex/8?id=29440
960
unsupported operand type(s) for +: 'int' and 'str'


,Product Brand,Product Name,Current Price,Old Price,Rating,Product URL
0,Cole Haan,Leather Cashmere Blend Lined Basic Tech Tip Gl...,Now EUR 73.30,EUR 97.73,3.5672,https://www.macys.com/shop/product/cole-haan-l...
1,Cole Haan,Leather Cashmere Blend Lined Basic Tech Tip Gl...,Now EUR 73.30,EUR 97.73,3.5672,https://www.macys.com/shop/product/cole-haan-l...
2,Charter Club,"Ribbed 100% Cashmere Scarf, Created for Macy's",Sale EUR 43.28,EUR 108.21,4.0848,https://www.macys.com/shop/product/charter-clu...
3,Charter Club,"Ribbed 100% Cashmere Scarf, Created for Macy's",Sale EUR 43.28,EUR 108.21,4.0848,https://www.macys.com/shop/product/charter-clu...
4,I.N.C. International Concepts,"Doris Sparkle Mesh Pouch, Created for Macy's",Sale EUR 35.67,EUR 79.28,4.6644,https://www.macys.com/shop/product/i.n.c.-inte...
...,...,...,...,...,...,...
965,Fanatics,Women's Navy St. Louis Blues Primary Logo Adju...,EUR 23.92,None,None,https://www.macys.com/shop/product/womens-navy...
966,Hudson Baby,Infant Girl Headband and Socks Giftset 6pc,EUR 19.94,None,None,https://www.macys.com/shop/product/hudson-baby...
967,Hudson Baby,Infant Girl Headband and Socks Giftset 6pc,EUR 19.94,None,None,https://www.macys.com/shop/product/hudson-baby...
968,Hudson Baby,Infant Girl Headband and Socks Giftset 6pc,EUR 22.93,None,5,https://www.macys.com/shop/product/hudson-baby...


In [28]:

data = data.drop_duplicates(subset=['Product URL'])
data = data.reset_index(drop=True)
data.to_csv('macy_data.csv')